# Machine Learning for Sepsis Pathway  with MLFlow

Setup Libraries and plotting defaults. 

In [1]:
import sys
import os
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
#import statsmodels.formula.api as smf
#from statsmodels.stats.anova import anova_lm


import mlflow

import re
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from pandasql import sqldf
import sqlalchemy
import pyodbc

import dfply as dfp
from dfply import *

import datetime
from datetime import *

import pyodbc
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics         import balanced_accuracy_score, precision_score, classification_report
from sklearn.metrics         import recall_score, f1_score, make_scorer, cohen_kappa_score

from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import sklearn

from sklearn import model_selection
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier

import forestci as fci

from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer, PowerTransformer
from sklearn.metrics import r2_score, classification_report, confusion_matrix, roc_auc_score, accuracy_score, balanced_accuracy_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
#from statsmodels.stats.outliers_influence import variance_inflation_factor
#from statsmodels.tools.tools import add_constant
import xgboost as xgb
from imblearn.over_sampling import SMOTENC
n_est=55

from hyperopt.pyll import scope
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK

pd.set_option('display.max_columns', None)


import Utilities
import importlib
importlib.reload(Utilities)

import MLUtilities
importlib.reload(MLUtilities)

from matplotlib.colors import to_rgba

plt.style.use('./CEIH.mplstyle')
color_dict = {0: to_rgba('#32C0D2', 1),
                1: to_rgba("#E0B165", 1),}

cmap_blended = sns.blend_palette(["#ADE6ED","#70D3E0","#32C0D2","#289AA8","#307078"], as_cmap=True)


import math
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1],idx-1
    else:
        return array[idx],idx
    
OtherValues = {'TriageCategory_2':'Triage Category 2', 'SoBFlag': 'Shortness of Breath', 'TriageCategory_4':'Triage Category 4',
               'TriageCategory_1':'Triage Category 1', 'TriageCategory_5':'Triage Category 5', 'TriageCategory_3':'Triage Category 3',
             'FirstGCSScoreAdult':'Glasgow Coma Scale',
               'FirstUrinalysisBlood': 'Urinalysis Blood', 'IndigenousStatusDescription_Not Aboriginal-TSI':  'Non Indiginous',
               'IndigenousStatusDescription_Not Stated':  'Indiginous Status not stated', 'FirstUrinalysisLeukocytes':'Urinalysis Leukocytes',
               'FirstPulseRateBPM':'Pulse Rate', 'FirstRespiration':'Respiration Rate', 'FirstSpO2':'O2 Saturation (%)',  'Gender_Male':'Sex - Male',
               'FirstLevelofConsciousness':'Level of Consciousness', 'IndigenousStatusDescription_Aboriginal and TSI':'Aboriginal and TSI',
               'IndigenousStatusDescription_TSI':'TSI','Gender_Indeterminate':'Sex - Indeterminate', 'Gender_Unknown':'Sex - Unknown',
               'FirstO2Flow':'O2 Flow '
               }


plotSettings = {
        #  ( bins, xmin, xmax, log/linear)
        'AGEONADMISSION':(50,20,110,'linear','Age (y)', False, 0,0),
        
        
        'FirstBloodGlucose':(50,0,100,'log','Blood Glucose [mmol/L]', False,0, 0),
        'FirstTemperatureDegreesC':(50,30,45,'log', r"Temperature [$^\circ$C]", True,35.5,38.1),
                'FirstWeightKg':(50,50,150,'log','Weight [kg]', False, 0,0),

        
        'FirstPainAssessment': (11,-0.5,10.5,'log','Pain Assessment', False, 0,0),
        'FirstBPSystolic': (50,50,250,'log', 'BP Systolic [mm Hg]',True, 100,170 ),
        'FirstBPDiastolic': (50,0,200,'log', 'BP Diastolic [mm Hg]', False,0,0),
        'FirstEstimatedGlomerularFiltrationRate': (50,0,100,'log',r"Estimated Glomerular Filtration Rate [mL/min/1.73m$^{2}$]", True, 60, 100),
        'FirstCreatinine': (50,0,800,'log',r"Creatinine - Serum [$\mu$mol/L]", True, 45, 110),
        'FirstAlbumin': (60,0,60,'log','Albumin  Level [g/L]', True, 30, 48),
        'FirstTotalBilirubin': (60,0,100,'log',r"Total Bilirubin Level [$\mu$mol/L]", True, 2, 24 ),
        'FirstAlkalinePhosphatase': (60,0,800,'log','Alkaline Phosphatase Level [U/L]',True, 30,110),
        'FirstAlanineAminotransferase': (60,0,700,'log','Alanine Aminotransferase Level [U/L]', True,0,55),
        'FirstAspartateAminotransferase': (60,0,700,'log','Aspartate Aminotransferase Level [U/L]', True, 0,45),
        'FirstGammaGlutamylTransferase': (60,0,700,'log','Gamma Glutamyl Transferase Level [U/L]', True, 0, 60),
        'FirstLactateDehydrogenase': (60,0,1200,'log','Lactate Dehydrogenase [U/L]', True, 120, 250),
        'FirstHaemoglobin': (50,10, 220,'log','Haemoglobin [g/L]', True, 115, 175),
        'FirstWhiteCellCount': (50,0, 50,'log',r"White Cell Count [$\times 10^{9}$/L]",True, 4,11),
        'FirstPlateletCount': (50,0, 1000,'log',r"Platelet Count [$\times 10^{9}$/L]",True,150,500),
        'FirstNeutrophils': (50,0, 50,'log',r"Absolute Neutrophil Count [$\times 10^{9}$/L]",True,1.80,7.50),
        'FirstDDimer': (40,0, 20,'log',r"D-Dimer [mg/L]",True,0,0.79),
        'FirstCreactiveprotein': (50,0, 600,'log',r"C-Reactive Protein [mg/L]",True, 0,8),
        'FirstTroponinT': (50,0, 600,'log',r"Troponin T Level [mg/L]",True,0,16),
        'FirstNTproBNP': (50,0, 40000,'log',r"NT-pro Brain Natriuretic Peptide [mg/L]",True,0,124),
        
        'FirstAnionGapVenous': (50,0, 50,'log',r"Anion Gap Venous [mmol/L]", True, 7, 17),
        'FirstAnionGapArterial': (50,0, 50,'log',r"Anion Gap Arterial [mmol/L]",  True, 7, 17),
        'FirstBaseExcessVenous': (50,-30, 30,'log',r"Base Excess Venous [mmol/L]", True, -3, 3),
        'FirstBaseExcessArterial': (50,-30, 30,'log',r"Base Excess Arterial [mmol/L]", True, -3, 3),
        'FirstBilirubinVenous': (60,0, 60,'log',r"Bilirubin Venous [$\mu$mol/L]", True, 2,24),
        'FirstBilirubinArterial': (60,0, 60,'log',r"Bilirubin Arterial [$\mu$mol/L]", True, 2,24),
        'FirstCarboxyhaemoglobinVenous': (50,0, 20,'log',r"Carboxyhaemoglobin Venous [%]", True, 0.3, 1.8),
        'FirstCarboxyhaemoglobinArterial': (50,0, 20,'log',r"Carboxyhaemoglobin Arterial [%]", True, 0.3, 1.8),
        'FirstChlorideDirectVenous': (50,50, 150,'log',r"Chloride Direct Venous [mmol/L]",False, 100,109),
        'FirstChlorideDirectArterial': (50,50, 150,'log',r"Chloride Direct Arterial [mmol/L]", True, 100,109),
        'FirstCreatinineVenous':(50,0,500,'log',r"Creatinine Venous [$\mu$mol/L]", True, 50, 120),
        'FirstCreatinineArterial':(50,0,500,'log',r"Creatinine Arterial [$\mu$mol/L]", True, 50,120),
        'FirstGlucoseVenous':(50,0,30,'log',r"Glucose  Venous [mmol/L]", False,0,0),
        'FirstGlucoseArterial':(50,0,30,'log',r"Glucose  Arterial [mmol/L]", True,2.6,5.6),
        'FirstIonised Calcium Venous':(50,0,2,'log',r"Ionised Calcium Venous [mmol/L]", True, 1.1, 1.3),
        'FirstIonised Calcium Arterial':(50,0,2,'log',r"Ionised Calcium Arterial [mmol/L]", True, 1.1, 1.3),
        'FirstLactateVenous':(50,0,30,'log',r"Lactate Venous [mmol/L]", True, 0.2, 2.0),
        'FirstLactateArterial':(50,0,30,'log',r"Lactate Arterial [mmol/L]", True, 0.2, 2.0),
        'FirstMethaemoglobinVenous': (20,0, 3,'log',r"Methaemoglobin Venous [%]", True, 0.4, 1.2),
        'FirstMethaemoglobinArterial': (20,0, 3,'log',r"Methaemoglobin Arterial [%]", True, 0.2,0.6),
        'FirstOxygenSaturationVenous': (50,0, 100,'log',r"Oxygen Saturation Venous [%]", False, 0,0),
        'FirstOxygenSaturationArterial': (50,0, 100,'log',r"Oxygen Saturation Arterial [%]", True,95, 99),
        'FirstOxyhaemoglobinVenous': (50,0, 100,'log',r"Oxyhaemoglobin Venous [%]", False, 0,0),
        'FirstOxyhaemoglobinArterial': (50,0, 100,'log',r"Oxyhaemoglobin Arterial [%]", False, 0,0),
        'FirstReducedHaemoglobinVenous': (50,0, 100,'log',r"Reduced Haemoglobin Venous [%]", False, 0,0),
        'FirstReducedHaemoglobinArterial': (50,0, 100,'log',r"Reduced Haemoglobin Arterial [%]", False, 0,0),
        'FirstTotalHaemoglobinVenous': (50,10, 220,'log','Total Haemoglobin Venous [g/L]', True, 115,180),
        'FirstTotalHaemoglobinArterial': (50,10, 220,'log','Total Haemoglobin Arterial [g/L]', True, 115,180), 
        'FirstpCO2Venous': (50,0,150,'log', 'pCO2 Venous [mm Hg]', True,41,51),
        'FirstpCO2Arterial': (50,0,150,'log', 'pCO2 Arterial [mm Hg]', True, 35, 45 ),
        'FirstpO2Venous': (50,0,200,'log', 'pO2 Venous [mm Hg]', True, 25,40),
        'FirstpO2Arterial': (50,0,200,'log', 'pO2 Arterial [mm Hg]', True,67, 108),
        'FirstpHVenous': (50,6.8,7.8,'log', 'pH Venous', True, 7.32,7.42),
        'FirstpHArterial': (50,6.8,7.8,'log', 'pH Arterial', True, 7.36, 7.44),
        'FirstPotassiumDirectVenous':(50,0,10,'log',r"Potassium Direct Venous [mmol/L]", False, 0,0),
        'FirstPotassiumDirectArterial':(50,0,10,'log',r"Potassium Direct Arterial [mmol/L]", True, 3.1, 4.2),
        'FirstSodiumDirectVenous':(50,100,180,'log',r"Sodium Direct Venous [mmol/L]", False, 0,0),
        'FirstSodiumDirectArterial':(50,100,180,'log',r"Sodium Direct Arterial [mmol/L]", True, 137, 145),}


Failed to import duecredit due to No module named 'duecredit'
c:\Users\ibertr02\venvs\ceihml\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (15.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [2]:
# Sepsis HeartFailure PE Pneumonia COPD UTI
# SepsisPneumonia
Diagnosis =   "Sepsis"
# Diagnosis = 'SepsisPneumonia'
# Diagnosis = 'Admission'
# Diagnosis = 'GenMed'

DiagnosisString=Diagnosis+'Flag'

# DataSet1 All observations 
# DataSet2 Not including Oxygen Flow 
# Dataset3  Dataset2 plus age and gender
#  Dataset4 3 plus restriction on TC to 3 4 and 5 
#  Dataset 5 with LoS > 6 hours. 
# Datset30 includes indiginous
#  60 remove < 18, include gender and not age... 


DataSet = 'DataSet3'

# MLModel = 'randomforest'

print(Diagnosis,DiagnosisString)

Sepsis SepsisFlag


In [3]:
start_date =  '2023-01-01'  #dates[Facility][0]
end_date   =  '2024-01-01' #dates[Facility][1]

select_start_date = '2023-01-01'

DataReasons = "Training"

In [4]:
Data_Storage_File = 'DataFiles\Emergency_IPInfo_Complete_Numeric_Numeric_{}_{}_{}_Stage_01.pkl'.format(start_date,end_date,DataReasons)
print(Data_Storage_File)


with open(Data_Storage_File, 'rb') as file:
    Emergency_IPInfo_Complete_Numeric = pd.read_pickle(file)

DataFiles\Emergency_IPInfo_Complete_Numeric_Numeric_2023-01-01_2024-01-01_Training_Stage_01.pkl


In [5]:
Emergency_IPInfo_Complete_Numeric = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION>17].copy()
importlib.reload(MLUtilities)
Emergency_IPInfo_Complete_Numeric = MLUtilities.setNumeric(Emergency_IPInfo_Complete_Numeric)

In [6]:
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(0,17),'age_range']='0-17'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(18,33),'age_range']='18-33'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(34,48),'age_range']='34-48'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(49,64),'age_range']='49-64'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(65,78),'age_range']='65-78'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(79,98),'age_range']='79-98'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(98,120),'age_range']='98+'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(121,200),'age_range']='Unknown'


Emergency_IPInfo_Complete_Numeric['IndiginousFlag'] = 0  # np.where((Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isna()|Emergency_IPInfo_Complete_Numeric.IndiginousStatus=='Not Aboriginal-TSI'),0,1)
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isin(['Aboriginal','Aboriginal and TSI', 'TSI']),'IndiginousFlag'] = 1

In [7]:
Emergency_IPInfo_Complete_Numeric.age_range.unique()

array(['65-78', '79-98', '34-48', '49-64', '98+', '18-33', 'Unknown'],
      dtype=object)

# Setup Machine Learning Information 



In [8]:
GenMedList = ['LMH-LGMED','LMH-LAMU',
              "MPH-MGMED", 
              "FMC-GENMED",
              "NHS-GENMED",
              "RAH-GENMED",
              "QEH-GENMED", 'QEH-GM-AMU'
              ]


# DiagnosisString='SepsisFlag'

if Diagnosis == "SepsisPneumonia":
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.SepsisFlag==1)|(Emergency_IPInfo_Complete_Numeric.PneumoniaFlag==1), DiagnosisString] = 1

if Diagnosis == 'Admission':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>24), DiagnosisString] = 1
    
    if DataSet == 'DataSet5':
        Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
        Emergency_IPInfo_Complete_Numeric.loc[ (Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>6), DiagnosisString] = 1


if Diagnosis == 'GenMed':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    for g in GenMedList:
        Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.GROUPCODE.str.contains(g))  & (Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>6), DiagnosisString] = 1
   


print("Diagnosis being investigated = {}".format(DiagnosisString))
print("Total Number of Cases = {}".format(len(Emergency_IPInfo_Complete_Numeric.index)))
print("Number of {} Cases ".format(DiagnosisString),len(Emergency_IPInfo_Complete_Numeric[Emergency_IPInfo_Complete_Numeric[DiagnosisString]==1]))


Diagnosis being investigated = SepsisFlag
Total Number of Cases = 277021
Number of SepsisFlag Cases  2733


In [9]:
if DataSet == 'DataSet4':
    df = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.TRIAGE_CATEGORY>2].copy()
else:
    df = Emergency_IPInfo_Complete_Numeric.copy()

df.name='{} diagnosis detected from full Diagnosis list, updated'.format(Diagnosis)
importlib.reload(MLUtilities)
df = MLUtilities.setDefaults(df)

In [10]:
AnalysisVariable =['FirstBPSystolic', 'FirstLevelofConsciousness',
       'FirstPulseRateBPM', 'FirstRespiration', 'FirstSpO2',
       'FirstTemperatureDegreesC', 
       DiagnosisString]

if DataSet == 'DataSet1':
       AnalysisVariable.append('FirstO2Flow')
elif DataSet == 'DataSet3':
       AnalysisVariable.append('GENDERCODE')
       AnalysisVariable.append('AGEONADMISSION')
elif (DataSet == 'DataSet4') or (DataSet == 'DataSet5'):
       AnalysisVariable.append('GENDERCODE')
       AnalysisVariable.append('AGEONADMISSION')
elif DataSet == 'DataSet30':
       AnalysisVariable.append('GENDERCODE')
       AnalysisVariable.append('AGEONADMISSION')
       AnalysisVariable.append('IndiginousFlag')
# elif DataSet == 'DataSet40':
#        AnalysisVariable.append('GENDERCODE')
#        AnalysisVariable.append('age_range')
       # AnalysisVariable.append('IndiginousFlag')
# elif DataSet == 'DataSet60':
#        AnalysisVariable.append('GENDERCODE')
#        # AnalysisVariable.append('age_range')


In [11]:
df_temp = df[AnalysisVariable].copy()
# df_temp[DiagnosisString] = df_temp[DiagnosisString].astype('category')
# df_temp['TriageCategory'] = df_temp['TriageCategory'].astype('category')
# df_temp['Gender'] = df_temp['Gender'].astype('category')
# df_temp['IndigenousStatusDescription'] = df_temp['IndigenousStatusDescription'].astype('category')                           
# df_temp['SoBFlag'] = df_temp['SoBFlag'].astype('category')
# df_temp.dropna(inplace=True)


if DataSet == 'DataSet3':
    df_temp['GENDERCODE'] = df_temp['GENDERCODE'].astype('category')

if DataSet == 'DataSet30':
    df_temp['GENDERCODE'] = df_temp['GENDERCODE'].astype('category')
    df_temp['IndiginousFlag'] = df_temp['IndiginousFlag'].astype('category')

if DataSet == 'DataSet40':
    df_temp['GENDERCODE'] = df_temp['GENDERCODE'].astype('category')
    # df_temp['IndiginousFlag'] = df_temp['IndiginousFlag'].astype('category')
    df_temp['age_range'] = df_temp['age_range'].astype('category')

# if DataSet == 'DataSet60':
#     df_temp['GENDERCODE'] = df_temp['GENDERCODE'].astype('category')
#     # df_temp['IndiginousFlag'] = df_temp['IndiginousFlag'].astype('category')
#     # df_temp['age_range'] = df_temp['age_range'].astype('category')


# SMOTE does not work if NAN is allowed so int instead of Int 
    
numeric_features = df_temp.select_dtypes(
        include=[ 'int64', 'Int64', 'int32']).columns

for key in numeric_features:
    df_temp[key] = df_temp[key].astype('int64')



In [12]:
y = df_temp[[DiagnosisString]]
X = df_temp[AnalysisVariable].drop(DiagnosisString, axis=1).copy()
x_vars = X.columns

## Variables included in model 

In [13]:
skew_vars = []

numeric_features = X.drop(skew_vars, axis=1).select_dtypes(
        include=['float64', 'int64']).columns

categorical_features = X.select_dtypes(
        include=['category', 'object']).columns

categorical_features,numeric_features

(Index(['GENDERCODE'], dtype='object'),
 Index(['FirstBPSystolic', 'FirstLevelofConsciousness', 'FirstPulseRateBPM',
        'FirstRespiration', 'FirstSpO2', 'FirstTemperatureDegreesC',
        'AGEONADMISSION'],
       dtype='object'))

In [14]:
numeric_transformer = Pipeline(steps=[
        ('scaler', MinMaxScaler()),  # StandardScaler #MinMaxScaler
        #('skew', FunctionTransformer(log_transform))
    ])

category_values = []
for i in categorical_features:
    print(i,pd.unique(df_temp[i]).tolist())
    category_values.append(pd.unique(df_temp[i]).tolist())

categorical_transformer = Pipeline(steps=[
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore',categories=category_values   ))])  # OrdinalEncoder() #OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_features),
            ('categorical', categorical_transformer, categorical_features)
        ])

def pip_f(reg):
    pip = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', reg)
    ])
    return(pip)

functions = {
        # 'decisiontree': DecisionTreeClassifier(class_weight='balanced', random_state=n_est),   #######class_weight='balanced' check this
        'randomforest': RandomForestClassifier(class_weight='balanced', random_state=n_est),
        # 'knn': KNeighborsClassifier(),
        # 'xgboost':xgb.XGBClassifier (class_weight='balanced', random_state=n_est)
        'xgboost':xgb.XGBClassifier ( random_state=n_est) # class_weight='balanced' not in xgboost??
    }

key = 'randomforest'
best_performance = {}

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

GENDERCODE ['Female', 'Male', 'Indeterminate', 'Unknown']


In [15]:
if len(categorical_features)>0:
    smote = SMOTENC(random_state=42, categorical_features=[X_train.columns.get_loc(c) for c in categorical_features])
    X_train, y_train= smote.fit_resample(X_train, y_train)

In [16]:
df.name='Sepsis diagnosis detected from full Diagnosis list, updated'

print(df.name)

Sepsis diagnosis detected from full Diagnosis list, updated


In [ ]:
mlflow.set_experiment("Iain Mucking Around")

In [ ]:
# mlflow.xgboost.autolog()

In [ ]:
print(len(X_train.index),len(y_train))


In [ ]:
# dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
# dtest = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
n_est = 52

with mlflow.start_run():
    params = {
        'random_state': n_est, 
        'max_depth': 20, 
        # 'n_estimators': 50,
        'learning_rate': 0.3,  # New 
        'subsample':1.,     # New
        # "num_class": 3,
        # "eval_metric": "mlogloss",
    }
    preprocessor = ColumnTransformer(
            transformers=[
                ('numeric', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, categorical_features)
            ])

    classifier = xgb.XGBClassifier(**params)

    model = Pipeline([
            ("preprocessor", preprocessor),
            ("classifier", classifier)
            ])

    model.fit(X_train,y_train)
    # # y_proba = model.predict(dtest)
    # y_pred = y_proba.argmax()
    # loss = log_loss(y_valid, y_proba)
    # acc = accuracy_score(y_valid, y_pred)

    predictions_valid = model.predict_proba(X_valid)
    ypreds = np.delete(predictions_valid,[0],1)

    fpr, tpr, thresholds = roc_curve(y_valid, ypreds)
    val,idx = find_nearest(tpr,0.85)
    threshold =   np.interp(0.85,tpr,thresholds)    #thresholds[idx]
    false_positive =  np.interp(0.85,tpr,fpr)      # fpr[idx]
    # redo predictions with new threshold 
    y_select_2 = np.where(ypreds<threshold,0,1)
    auc_score = roc_auc_score(y_valid, ypreds, multi_class="ovr", average="weighted")
    auc_score2 = roc_auc_score(y_valid, ypreds)
    accuracy  =accuracy_score(y_valid, y_select_2)
    balanced_accuracy = balanced_accuracy_score (y_valid, y_select_2)
    recall  =recall_score(y_valid, y_select_2)
    precision = precision_score(y_valid, y_select_2)
    f1 = f1_score(y_valid, y_select_2)

    val,idx = find_nearest(tpr,0.85)
    threshold2 = thresholds[idx]
    false_positive2 = fpr[idx]

    mlflow.log_metrics({'loss': false_positive, "falsePositive":false_positive,
              "accuracy":accuracy,
              "precision":precision,
              "recall":recall,
              "precision":precision,
              "f1":f1,
              "auc":auc_score,
              "falsePositive2":false_positive2,
              "threshold":threshold,})

## Hyperopt test 

In [27]:
mlflow.set_experiment("Iain Hyperopt Test")

2024/11/06 10:22:42 INFO mlflow.tracking.fluent: Experiment with name 'Iain Hyperopt Test' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///e:/105GenMed-ML-OLd/GenMed105-DxPrediction/SnowFlakeVersion/mlruns/209495169642519356', creation_time=1730850762588, experiment_id='209495169642519356', last_update_time=1730850762588, lifecycle_stage='active', name='Iain Hyperopt Test', tags={}>

In [28]:
space4xgbsepsis = {
    # 'random_state': n_est, 
    'max_depth': hp.choice('max_depth', np.arange(1,30, 1, dtype=int)), 
    'n_estimators': hp.choice('n_estimators', np.arange(1, 200, 1, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),  # New 
    'subsample':hp.uniform('subsample', 0.5, 1.),     # New
    'seed':hp.choice('seed',[0,7,42])}

In [29]:
def objective (params):
    with mlflow.start_run():
        
        
        
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        
        classifier = xgb.XGBClassifier(**params)

        model = Pipeline([
            ("preprocessor", preprocessor),
            ("classifier", classifier)
        ])
        
        model.fit(X_train, np.ravel(y_train,order="c"))
        predictions_valid = model.predict_proba(X_valid)
        ypreds = np.delete(predictions_valid,[0],1)
    
        fpr, tpr, thresholds = roc_curve(y_valid, ypreds)
        val,idx = find_nearest(tpr,0.85)
        threshold =   np.interp(0.85,tpr,thresholds)    #thresholds[idx]
        false_positive =  np.interp(0.85,tpr,fpr)      # fpr[idx]
        # redo predictions with new threshold 
        y_select_2 = np.where(ypreds<threshold,0,1)
        auc_score = roc_auc_score(y_valid, ypreds, multi_class="ovr", average="weighted")
        auc_score2 = roc_auc_score(y_valid, ypreds)
        accuracy  =accuracy_score(y_valid, y_select_2)
        balanced_accuracy = balanced_accuracy_score (y_valid, y_select_2)
        recall  =recall_score(y_valid, y_select_2)
        precision = precision_score(y_valid, y_select_2)
        f1 = f1_score(y_valid, y_select_2)
    
        scores = {"falsePositive":false_positive,
              "accuracy":accuracy,
              "precision":precision,
              "recall":recall,
              "precision":precision,
              "f1":f1,
              "auc":auc_score,
              "auc2":auc_score2,
            #   "falsePositive2":false_positive2,
              "threshold":threshold,
             }
    
        mlflow.log_metrics({'loss': false_positive, "falsePositive":false_positive,
            "accuracy":accuracy,
            "precision":precision,
            "recall":recall,
            "precision":precision,
            "f1":f1,
            "auc":auc_score,
            # "falsePositive2":false_positive2,
            "threshold":threshold,})
    
    return {'status': STATUS_OK, 'loss': false_positive, 'model': model, 'attachments':scores} 


In [30]:
n_trials = 1000

trialsXGBoost = Trials()

best_params_XGB = fmin(fn = objective,
                    space = space4xgbsepsis,
                    algo = tpe.suggest,
                    max_evals = n_trials, 
                    trials = trialsXGBoost
                    )

  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 1000/1000 [2:12:47<00:00,  7.97s/trial, best loss: 0.20494640708738923] 


In [60]:
best_params_XGB

{'learning_rate': 0.058495033316467944,
 'max_depth': 0,
 'n_estimators': 69,
 'seed': 1,
 'subsample': 0.7575856608007053}

In [62]:
pointer = np.argmin([r['loss'] for r in trialsXGBoost.results])
scoreXGBoost = [t['result']['loss'] for t in trialsXGBoost.trials]
best_model_XGBoost=trialsXGBoost.results[np.argmin([r['loss'] for r in trialsXGBoost.results])]['model']

In [63]:
pointer

577

In [44]:
description = 'A xgBoost model on real data'
tags = {'version': 'v0.1', 'type': 'demo'}
experiment_name = 'Sklearn Pipeline Tutorial'
registered_model_name = 'xgb - dataset3'
from pathlib import Path



In [45]:
mlflow.create_experiment(name=experiment_name,
                         tags=tags,
                         artifact_location=Path.cwd().joinpath('mlruns').as_uri())

'439434785447189246'

In [46]:
mlflow.set_experiment(experiment_name=experiment_name)


<Experiment: artifact_location='file:///e:/105GenMed-ML-OLd/GenMed105-DxPrediction/SnowFlakeVersion/mlruns', creation_time=1730864940755, experiment_id='439434785447189246', last_update_time=1730864940755, lifecycle_stage='active', name='Sklearn Pipeline Tutorial', tags={'type': 'demo', 'version': 'v0.1'}>

In [66]:
scores = {}
i = pointer
varStrings = ["falsePositive",  "threshold","accuracy", "precision", "recall","precision","f1", "auc"]
for c in varStrings:
    # print(c)
    key = "ATTACH::{}::{}".format(i,c)
    scores[c] = trialsXGBoost.attachments[key]
    # scores["i"] = i

scores

{'falsePositive': 0.20494640708738923,
 'threshold': 0.4606784999370575,
 'accuracy': 0.7955960653370634,
 'precision': 0.039637792584999144,
 'recall': 0.8482632541133455,
 'f1': 0.07573655431322941,
 'auc': 0.89173515494183}

In [68]:
# best_model_XGBoost.get_params()
signature  = mlflow.models.infer_signature(best_model_XGBoost.predict(X_train))
# mlflow.data.from_pandas(df_temp,targets=DiagnosisString).to_dict()
best_params = best_params_XGB
best_metrics_dict = scores

In [75]:
# with mlflow.start_run(run_name='Parent Run', description=description) as run:
#     run_id = run.info.run_id

mlflow.log_params(best_params)
mlflow.log_metrics(best_metrics_dict)
mlflow.sklearn.log_model(best_model_XGBoost, 'best_model',
                            # signature=signature,
                            input_example=X_train.head(5),
                            registered_model_name=registered_model_name)
    

c:\Users\ibertr02\venvs\ceihml\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/11/08 12:44:55 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to enforce schema of data '   FirstBPSystolic  FirstLevelofConsciousness  FirstPulseRateBPM  \

In [77]:
stage = 'staging'
client = mlflow.MlflowClient()
client.transition_model_version_stage(name=registered_model_name,version=3,stage=stage)

C:\Users\ibertr02\AppData\Local\Temp\ipykernel_42560\283172245.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name=registered_model_name,version=3,stage=stage)


<ModelVersion: aliases=[], creation_timestamp=1731032102236, current_stage='Staging', description=None, last_updated_timestamp=1731038897712, name='xgb - dataset3', run_id='fc511fc624a8423eb45c77ad0a37b9cd', run_link=None, source='file:///e:/105GenMed-ML-OLd/GenMed105-DxPrediction/SnowFlakeVersion/mlruns/fc511fc624a8423eb45c77ad0a37b9cd/artifacts/best_model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [78]:
model = mlflow.pyfunc.load_model(model_uri=f'models:/{registered_model_name}/{stage}')

c:\Users\ibertr02\venvs\ceihml\Lib\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


In [ ]:
model.predict(X_valid.head(100))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
xgbValues = []
varStrings = ["falsePositive",  "threshold","accuracy", "precision", "recall","precision","f1", "auc","auc2"]
scores = {}
i=0
for i in range(n_trials):
    scores = {}
    # if i < 2:
    scores['max_depth'] = trialsXGBoost.trials[i]['misc']['vals']['max_depth'][0]+1
    scores['n_estimators'] = trialsXGBoost.trials[i]['misc']['vals']['n_estimators'][0]+1
    scores['learning_rate'] = trialsXGBoost.trials[i]['misc']['vals']['learning_rate'][0]+1
    scores['subsample'] = trialsXGBoost.trials[i]['misc']['vals']['subsample'][0]+1
    scores['seed'] = trialsXGBoost.trials[i]['misc']['vals']['seed'][0]

    

        # 
        # 'n_estimators'
    for c in varStrings:
        # print(c)
        key = "ATTACH::{}::{}".format(i,c)
        scores[c] = trialsXGBoost.attachments[key]
        scores["i"] = i
    xgbValues.append(scores)

In [64]:
trialsXGBoost.trials[10]['misc']['attach']

KeyError: 'attach'

In [55]:
trialsXGBoost.trials[10]

{'state': 2,
 'tid': 10,
 'spec': None,
 'result': {'status': 'ok',
  'loss': 0.3338984286703854,
  'model': Pipeline(steps=[('preprocessor',
                   ColumnTransformer(transformers=[('numeric',
                                                    Pipeline(steps=[('scaler',
                                                                     MinMaxScaler())]),
                                                    Index(['FirstBPSystolic', 'FirstLevelofConsciousness', 'FirstPulseRateBPM',
         'FirstRespiration', 'FirstSpO2', 'FirstTemperatureDegreesC',
         'AGEONADMISSION'],
        dtype='object')),
                                                   ('categorical',
                                                    Pipeline(steps=[('encoder',
                                                                     OneHotEncoder(categorie...
                                 feature_types=None, gamma=None, grow_policy=None,
                                 importance_type=N

In [40]:
trials_XGB_df = pd.DataFrame.from_dict(xgbValues)

In [41]:
trials_XGB_df

,max_depth,n_estimators,learning_rate,subsample,seed,falsePositive,i,threshold,accuracy,precision,recall,f1,auc,auc2
0,16,163,1.173579,1.867897,3,0.337362,0,0.001433,0.664471,0.024458,0.848263,0.047546,0.833664,0.833664
1,10,12,1.089087,1.935710,1,0.387473,1,0.265316,0.614854,0.021363,0.848263,0.041676,0.826050,0.826050
2,15,172,1.173232,1.999207,2,0.372762,2,0.001815,0.629420,0.022187,0.848263,0.043243,0.823335,0.823335
3,4,92,1.080891,1.942103,3,0.267235,3,0.250335,0.733905,0.030680,0.848263,0.059218,0.878320,0.878320
4,2,150,1.284513,1.882644,3,0.262514,4,0.154030,0.738580,0.031214,0.848263,0.060213,0.876990,0.876990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,197,1.070433,1.569413,2,0.245394,995,0.384214,0.755582,0.033326,0.848263,0.064133,0.890840,0.890840
996,16,189,1.033319,1.676919,2,0.343268,996,0.020023,0.658623,0.024048,0.848263,0.046769,0.845447,0.845447
997,2,124,1.020452,1.751205,2,0.238697,997,0.420887,0.763469,0.034337,0.846435,0.065997,0.890908,0.890908
998,7,47,1.039398,1.639796,1,0.313624,998,0.273289,0.688043,0.026267,0.848263,0.050955,0.867825,0.867825
